### Импорт, предварительные штуки

Моделька

In [4]:
from deeppavlov import build_model, configs

model = build_model(configs.ner.ner_rus_bert, download=True)

2025-03-04 19:40:08.683 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_rus_bert_torch_new.tar.gz download because of matching hashes
c:\Users\ivanr\Documents\cod\.venv\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-03-04 19:40:42.77 WARNING in 'deeppavlov.core.models.torch_model'['torch_model'] at line 96: Unable to place component TorchTransformersSequenceTagger on GPU, since no CUDA GPUs are available. Using CPU.


Загрузка и очистка текста

In [ ]:
import re
text = ""
with open("Pelevin_Generation_p.txt", 'r', encoding="UTF-8") as f:
    text = f.read()

In [6]:
from pymystem3 import Mystem

m = Mystem()

### NER

Функции

In [16]:
def split_text(text, max_len=400): # Функция для разбиения текста на параграфы для быстрой обработки
    paragraphs = text.split("\n")
    chunks = []
    cur_chunk = ""
    for paragraph in paragraphs:
        if len(paragraph) > max_len:
            if cur_chunk:
                chunks.append(cur_chunk)
                cur_chunk = ""
            for i in range(0, len(paragraph), max_len):
                chunks.append(paragraph[i:i+max_len])
        else:
            if (len(cur_chunk) + len(paragraph) <= max_len):
                cur_chunk += " " + paragraph if cur_chunk else paragraph
            else:
                if(cur_chunk):
                    chunks.append(cur_chunk)
                cur_chunk = paragraph
    if cur_chunk:
        chunks.append(cur_chunk)
    return chunks

def lemmatizer(text): # Функция для лемматизации каждой отдельной сущности
    lemmas = m.lemmatize(text)
    return "".join(lemmas).strip()

def merge_entities(entities): # Функция для мерджа смежных токенов в единую сущность
    merged = []
    for elem in entities:
        token, label = elem
        if label.startswith("B"):
            merged.append([token, label[2:]])
        else:
            if merged:
                merged[-1][0] += " " + token
            else:
                merged.append([token, label[2:]])
    return merged

def is_valid_entity(entity): # Функция для фильтрации от ненужных сущностей
    if len(entity) < 3:
        return False
    if re.search(r'\d', entity):
        return False
    return True

def cleaner(entity): # Функция для отсчистки сущностей от лишних символов
    cleaned = re.sub(r"[^A-Za-zА-Яа-яЁё\s]", "", entity)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned


Извлечение

In [8]:
chunks = split_text(text)
res = dict()
for chunk in chunks: # Создание массива словарей, где в каждом словаре сущности чанка -- ключ, их типы -- значение
    chunk_fitted = model([chunk])
    chunk_keys = tuple(chunk_fitted[0][0])
    chunk_values = tuple(chunk_fitted[1][0])
    res[chunk_keys] = chunk_values

"Причёсывание" списка сущностей

In [ ]:
from collections import Counter

ents0 = []
for key in res.keys(): # Массив с локациями, организациями, персонажами 
    for i in range(len(key)):
        if res[key][i] != 'O':
            cleand_key = cleaner(key[i])
            ents0.append([cleand_key, res[key][i]])

ents1 = merge_entities(ents0) # Объединям разделённые на части сущности


# Список лемматизированных сущностей
ents_lemmatized = [[lemmatizer(ent[0]), ent[1]] for ent in ents1]

ents_val_only = [ent[0] for ent in ents_lemmatized]
freq = Counter(ents_val_only) # Создаём список особо частотных сущностей
threshold = 5
exclude = {entity for entity, count in freq.items() if count >= threshold}

ents_clear = [ent for ent in ents_lemmatized if ent[0] not in exclude and is_valid_entity(ent[0])] # Убираем персонажей произведения и всякий мусор

Персонажи

In [ ]:
pers = []
for ent in ents_clear:
    if ent[1] == "PER" and len(ent[0]) >= 4:
        pers.append(ent[0])

pers_unique = set(pers)
pers_unique_list = list(pers_unique)